### Import libraries

In [1]:
import torch
import torch.nn as nn                   # all neural network modules, nn.Linear, nn.Conv2d, BatchNorm, Loss functions
import torch.nn.functional as F         # parametersless functions, like (some) activation functions
import torch.optim as optim             # all optimization algorithms, SGD, Adam, etc
from torch.utils.data import DataLoader # gives easier dataset management and creates mini batches
import torchvision
from torchvision import datasets        # has standard datasets we can import in a nice and easy way
from torchvision import transforms      # transformations we can perform on our dataset (data processing)

import matplotlib.pyplot as plt

print("Pytorch version:", torch.__version__)

Pytorch version: 1.11.0


### Set device

In [ ]:
# device config
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

### Hyperparameters

In [ ]:
in_channels = 1    # 28x28 = 784, size of MNIST images (grayscale)
hidden_size = 100
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epochs = 2

### Create Network

In [ ]:
class CNN(nn.Module):
    def __init__(self, in_channels = 1, num_classes = 10):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=8, kernel_size=(3,3), stride=(1,1), padding=(1,1))
        self.pool = nn.MaxPool2d(kernel_size=(2,2), stride=(2,2))
        self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=(3,3), stride=(1,1), padding=(1,1))
        self.fc1 = nn.Linear(16*7*7, num_classes)

    def forward(self, x):
        output = self.conv1(x)
        output = F.relu(output)
        output = self.pool(output)
        output = self.conv2(output)
        output = F.relu(output)
        output = self.pool(output)
        output = output.reshape(output.shape[0], -1)
        output = self.fc1(output)
        return output